# ALOS-2

Downloading ALOS-2 mosaic and applying Enhanced Lee filter.

https://www.eorc.jaxa.jp/ALOS/en/dataset/fnf_e.htm

In [ ]:
import subprocess
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio

from vegmapper import alos2
from vegmapper import pathurl

plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = [16, 12]

## User Inputs

In [ ]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = sitename

# AOI file
aoifile = f'{proj_dir}/ucayali_boundary.geojson'

# Reference tiles
tiles = f'{proj_dir}/{sitename}_tiles.geojson'

# Year of interest
year = 2020

## Download ALOS/ALOS-2 Mosaic Tiles

### `alos2.download_tiles`

```
alos2.download_tiles(proj_dir, aoifile, year, quiet=True)
```

Paremeters:

|Paremeters|Description|Required|Default|
|----|----|----|----|
|proj_dir|Project directory|Yes||
|aoifile|AOI file in vector-based spatial data format (shapefile, GeoJSON, ...)|Yes||
|year|Year|Yes||
|quiet|Turn off wget's output.|No|True|

In [ ]:
alos2.download_tiles(proj_dir, aoifile, year)

## Process ALOS/ALOS-2 Mosaic Tiles

### `alos2.proc_tiles`

```
alos2.proc_tiles(proj_dir, year, filter_win_size=5, filter_num_looks=1)
```

Paremeters:

|Paremeters|Description|Required|Default|
|----|----|----|----|
|proj_dir|Project directory|Yes||
|year|Year|Yes||
|filter_win_size|Window size of Enhanced Lee filter|No|5|
|filter_num_looks|Number of looks used in Enhanced Lee filter|No|1|

In [ ]:
alos2.proc_tiles(proj_dir, year)

In [ ]:
# Read and display HH
with rasterio.open(f'{proj_dir}/ALOS-2/mosaic/{year}/alos2_mosaic_{year}_HH.vrt') as dset:
    HH = dset.read(1)
plt.imshow(HH, vmin=0, vmax=0.5, cmap='Greys')

## Fill the gaps with data of prior years
As shown above that ALOS-2 Mosaic data may have gaps where no data were acquired, we will use the data of prior years to fill these gaps.

In [ ]:
alos2.download_tiles(proj_dir, aoifile, 2019)

In [ ]:
alos2.proc_tiles(proj_dir, 2019)

In [ ]:
# Read and display HH of year 2019
year = 2019
with rasterio.open(f'{proj_dir}/ALOS-2/mosaic/{year}/alos2_mosaic_{year}_HH.vrt') as dset:
    HH = dset.read(1)
plt.imshow(HH, vmin=0, vmax=0.5, cmap='Greys')

With ALOS-2 Mosaic data of both 2020 and 2019, we virtually merge them into one virtual raster (VRT) by running the cell below. The 2020 data will take precedence over the 2019 data (by placing 2020 VRT file at the end of the list for gdalbuildvrt program).

In [ ]:
for layer in ['HH', 'HV', 'INC']:
    vrt_2019 = f'{proj_dir}/ALOS-2/mosaic/2019/alos2_mosaic_2019_{layer}.vrt'
    vrt_2020 = f'{proj_dir}/ALOS-2/mosaic/2020/alos2_mosaic_2020_{layer}.vrt'
    vrt_combined = f'{proj_dir}/ALOS-2/mosaic/alos2_mosaic_2020-2019_{layer}.vrt'
    !gdalbuildvrt -overwrite -srcnodata nan -vrtnodata nan $vrt_combined $vrt_2019 $vrt_2020

In [ ]:
with rasterio.open(f'{proj_dir}/ALOS-2/mosaic/alos2_mosaic_2020-2019_HH.vrt') as dset:
    HH = dset.read(1)
plt.imshow(HH, vmin=0, vmax=0.5, cmap='Greys')

## Warp to reference tiles virtually

In [ ]:
gdf_tiles = gpd.read_file(tiles)
t_epsg = gdf_tiles.crs.to_epsg()
for i in gdf_tiles.index:
    h = gdf_tiles['h'][i]
    v = gdf_tiles['v'][i]
    m = gdf_tiles['mask'][i]
    g = gdf_tiles['geometry'][i]

    if m == 0:
        continue
    
    for var in ['HH', 'HV', 'INC']:
        src_vrt = f'{proj_dir}/ALOS-2/mosaic/alos2_mosaic_2020-2019_{var}.vrt'
        dst_vrt = f'{proj_dir}/ALOS-2/mosaic/L-{var}-2020-2019-h{h}v{v}.vrt'
        cmd = (f'gdalwarp -overwrite '
               f'-t_srs EPSG:{t_epsg} -et 0 '
               f'-te {g.bounds[0]} {g.bounds[1]} {g.bounds[2]} {g.bounds[3]} '
               f'-tr 30 30 '
               f'-ot Float32 -wt Float32 '
               f'-dstnodata nan '
               f'-r average '
               f'-co COMPRESS=LZW '
               f'{src_vrt} {dst_vrt}')
        subprocess.check_call(cmd, shell=True)

In [ ]:
with rasterio.open(f'{proj_dir}/ALOS-2/mosaic/L-HH-2020-2019-h1v1.vrt') as dset:
    HH = dset.read(1)
plt.imshow(HH, vmin=0, vmax=0.5, cmap='Greys')

## Download ALOS-2 FBD data from SERVIR GS bucket (OPTIONAL)

In [ ]:
src_dir = pathurl.PathURL('gs://samz/alos2/fbd')
dst_dir = pathurl.PathURL(f'{proj_dir}/ALOS-2')
pathurl.copy(src_dir, dst_dir, overwrite=True)

In [ ]:
with rasterio.open(f'{proj_dir}/ALOS-2/fbd/HH.vrt') as dset:
    HH = dset.read(1)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 16]
plt.imshow(HH, vmin=0, vmax=0.5, cmap='Greys')